In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import re
import warnings
warnings.filterwarnings(action='ignore')
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloadin

True

In [3]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ybsession/0210 KoNLPyWordcloud/train.csv')
data.head()

,id,document,label
0,1,영상이나 음악이 이쁘다 해도 미화시킨 불륜일뿐,0
1,2,히치콕이 이 영화를 봤다면 분명 박수를 쳤을듯...,1
2,3,괜찮은 음악영화가 또 나왔군요!!! 따뜻한 겨울이 될 것 같아요~,1
3,4,아무래도 20년도지난작품이라 지금보기는너무유치하다,0
4,5,지금까지의 영화들이 그랬듯. 이 영화역시 일본에 대한 미화는 여전하다.,0


In [4]:
data.isna().sum()

id          0
document    0
label       0
dtype: int64

학습/검증 셋 분리

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
train, val = train_test_split(data)
train.reset_index(inplace=True)
val.reset_index(inplace=True)

In [12]:
train

,index,id,document,label
0,3178,3179,"아나,, 실화고 나발이고 머 이런 ㅡㅡ;",0
1,4059,4060,"황정민의 찰진 액션, 유아인의 똘끼, 마동석의 아트박스가 기억에 남는 영화",1
2,3257,3258,배우들 연기 죽이고~ 음악좋고!! 여운이 많이 남는 영화!!,1
3,4161,4162,"재밌구먼!! 난 이거 잊을수없어,ㅋㅋ",1
4,1053,1054,진짜뭐햐는 짓인지...스토리가 억지스럽고 자연스럽지못하고어처구니가 없네요,0
...,...,...,...,...
3745,3027,3028,영화의지루함보다더참기힘들었던건나를짜증나게만들던여주인공의얼굴과목소리였다,0
3746,3338,3339,아무생각없이 봤는데 가슴에 남는영화ㅠ ㅠ,1
3747,1723,1724,감독이 누군지 정말 궁금하다....,0
3748,4826,4827,"그다지 자극적이지 않은, 3류 성인 드라마를 보는 느낌",0


데이터 정제

In [13]:
train['preprocessed'] = train['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
train[:5]

,index,id,document,label,preprocessed
0,3178,3179,"아나,, 실화고 나발이고 머 이런 ㅡㅡ;",0,아나 실화고 나발이고 머 이런 ㅡㅡ
1,4059,4060,"황정민의 찰진 액션, 유아인의 똘끼, 마동석의 아트박스가 기억에 남는 영화",1,황정민의 찰진 액션 유아인의 똘끼 마동석의 아트박스가 기억에 남는 영화
2,3257,3258,배우들 연기 죽이고~ 음악좋고!! 여운이 많이 남는 영화!!,1,배우들 연기 죽이고 음악좋고 여운이 많이 남는 영화
3,4161,4162,"재밌구먼!! 난 이거 잊을수없어,ㅋㅋ",1,재밌구먼 난 이거 잊을수없어ㅋㅋ
4,1053,1054,진짜뭐햐는 짓인지...스토리가 억지스럽고 자연스럽지못하고어처구니가 없네요,0,진짜뭐햐는 짓인지스토리가 억지스럽고 자연스럽지못하고어처구니가 없네요


In [14]:
train['preprocessed'] = train['preprocessed'].str.replace(" +", " ")
train[:5]

,index,id,document,label,preprocessed
0,3178,3179,"아나,, 실화고 나발이고 머 이런 ㅡㅡ;",0,아나 실화고 나발이고 머 이런 ㅡㅡ
1,4059,4060,"황정민의 찰진 액션, 유아인의 똘끼, 마동석의 아트박스가 기억에 남는 영화",1,황정민의 찰진 액션 유아인의 똘끼 마동석의 아트박스가 기억에 남는 영화
2,3257,3258,배우들 연기 죽이고~ 음악좋고!! 여운이 많이 남는 영화!!,1,배우들 연기 죽이고 음악좋고 여운이 많이 남는 영화
3,4161,4162,"재밌구먼!! 난 이거 잊을수없어,ㅋㅋ",1,재밌구먼 난 이거 잊을수없어ㅋㅋ
4,1053,1054,진짜뭐햐는 짓인지...스토리가 억지스럽고 자연스럽지못하고어처구니가 없네요,0,진짜뭐햐는 짓인지스토리가 억지스럽고 자연스럽지못하고어처구니가 없네요


토큰화

In [15]:
# Example
text = "Hello, nice to meet you. What's your name? Have a nice day! See you soon."

In [16]:
from nltk.tokenize import sent_tokenize

print('문장 토큰화 결과 ==>',sent_tokenize(text))

문장 토큰화 결과 ==> ['Hello, nice to meet you.', "What's your name?", 'Have a nice day!', 'See you soon.']


In [17]:
from nltk.tokenize import word_tokenize

print('단어 토큰화 결과 ==>', word_tokenize(text))

단어 토큰화 결과 ==> ['Hello', ',', 'nice', 'to', 'meet', 'you', '.', 'What', "'s", 'your', 'name', '?', 'Have', 'a', 'nice', 'day', '!', 'See', 'you', 'soon', '.']


In [19]:
!pip install konlpy

     |████████████████████████████████| 19.4 MB 4.2 MB/s 
     |████████████████████████████████| 448 kB 49.7 MB/s 


In [20]:
from konlpy.tag import Okt 

han_sentence = "오늘도 열심히 코딩을 해볼까요? 같이 힘내서 자연어 처리 고수가 됩시다! ㅎㅎ"
okt = Okt()
print("한국어 형태소 분석 결과(어간 추출X) ==>", okt.morphs(han_sentence, stem = False))
print("한국어 형태소 분석 결과(어간 추출O) ==>", okt.morphs(han_sentence, stem = True))

한국어 형태소 분석 결과(어간 추출X) ==> ['오늘', '도', '열심히', '코딩', '을', '해볼까', '요', '?', '같이', '힘내서', '자연어', '처리', '고수', '가', '됩시다', '!', 'ㅎㅎ']
한국어 형태소 분석 결과(어간 추출O) ==> ['오늘', '도', '열심히', '코딩', '을', '해보다', '요', '?', '같이', '힘내다', '자연어', '처리', '고수', '가', '되다', '!', 'ㅎㅎ']


In [21]:
tokenized = []
for sentence in train['preprocessed']:
  tokens = okt.morphs(sentence, stem = True)
  tokenize = " ".join(tokens)
  tokenized.append(tokenize)
train['tokenized_stem'] = pd.DataFrame(tokenized)

train.head()

,index,id,document,label,preprocessed,tokenized_stem
0,3178,3179,"아나,, 실화고 나발이고 머 이런 ㅡㅡ;",0,아나 실화고 나발이고 머 이런 ㅡㅡ,아 나 실화 고 나발 이고 머 이렇다 ㅡㅡ
1,4059,4060,"황정민의 찰진 액션, 유아인의 똘끼, 마동석의 아트박스가 기억에 남는 영화",1,황정민의 찰진 액션 유아인의 똘끼 마동석의 아트박스가 기억에 남는 영화,황정민 의 찰지다 액션 유아인 의 똘끼 마동석 의 아트 박스 가 기억 에 남다 영화
2,3257,3258,배우들 연기 죽이고~ 음악좋고!! 여운이 많이 남는 영화!!,1,배우들 연기 죽이고 음악좋고 여운이 많이 남는 영화,배우 들 연기 죽 이고 음악 좋다 여운 이 많이 남다 영화
3,4161,4162,"재밌구먼!! 난 이거 잊을수없어,ㅋㅋ",1,재밌구먼 난 이거 잊을수없어ㅋㅋ,재밌다 난 이 거 잊다 없다 ㅋㅋ
4,1053,1054,진짜뭐햐는 짓인지...스토리가 억지스럽고 자연스럽지못하고어처구니가 없네요,0,진짜뭐햐는 짓인지스토리가 억지스럽고 자연스럽지못하고어처구니가 없네요,진짜뭐햐 는 짓 인지 스토리 가 억지스럽다 자연스럽다 어처구니 가 없다


품사 태깅

In [22]:
print(okt.pos("오늘도 열심히 재밌는 코딩을 해볼까? 같이 힘내서 자연어 처리 고수들이 됩시다! ㅎㅎ"))

[('오늘', 'Noun'), ('도', 'Josa'), ('열심히', 'Adverb'), ('재밌는', 'Adjective'), ('코딩', 'Noun'), ('을', 'Josa'), ('해볼까', 'Verb'), ('?', 'Punctuation'), ('같이', 'Adverb'), ('힘내서', 'Verb'), ('자연어', 'Noun'), ('처리', 'Noun'), ('고수', 'Noun'), ('들', 'Suffix'), ('이', 'Josa'), ('됩시다', 'Verb'), ('!', 'Punctuation'), ('ㅎㅎ', 'KoreanParticle')]


In [23]:
print(okt.pos("이것은 1점이 아니다 11점을 주고싶은 내 간절한 마음이다"))

[('이', 'Determiner'), ('것', 'Noun'), ('은', 'Josa'), ('1', 'Number'), ('점', 'Noun'), ('이', 'Josa'), ('아니다', 'Adjective'), ('11', 'Number'), ('점', 'Noun'), ('을', 'Josa'), ('주고싶은', 'Verb'), ('내', 'Noun'), ('간절한', 'Adjective'), ('마음', 'Noun'), ('이다', 'Josa')]


In [24]:
print(okt.nouns("오늘도 열심히 재밌는 코딩을 해볼까? 같이 힘내서 자연어 처리 고수들이 됩시다! ㅎㅎ"))

['오늘', '코딩', '자연어', '처리', '고수']


In [26]:
main_pos = []
for sentence in train['document']:
  pos = okt.pos(sentence)
  main_words = [word_pos[0] for word_pos in pos if word_pos[1] in ("Noun", "Adverb", "Adjective", "Verb")]
  main_words_str = " ".join(main_words)
  main_pos.append(main_words_str)
train["main_pos"] = pd.DataFrame(main_pos)

train.head()

,index,id,document,label,preprocessed,tokenized_stem,main_pos
0,3178,3179,"아나,, 실화고 나발이고 머 이런 ㅡㅡ;",0,아나 실화고 나발이고 머 이런 ㅡㅡ,아 나 실화 고 나발 이고 머 이렇다 ㅡㅡ,아 실화 나발 머 이런
1,4059,4060,"황정민의 찰진 액션, 유아인의 똘끼, 마동석의 아트박스가 기억에 남는 영화",1,황정민의 찰진 액션 유아인의 똘끼 마동석의 아트박스가 기억에 남는 영화,황정민 의 찰지다 액션 유아인 의 똘끼 마동석 의 아트 박스 가 기억 에 남다 영화,황정민 찰진 액션 유아인 똘끼 마동석 아트 박스 기억 남는 영화
2,3257,3258,배우들 연기 죽이고~ 음악좋고!! 여운이 많이 남는 영화!!,1,배우들 연기 죽이고 음악좋고 여운이 많이 남는 영화,배우 들 연기 죽 이고 음악 좋다 여운 이 많이 남다 영화,배우 연기 죽 음악 좋고 여운 많이 남는 영화
3,4161,4162,"재밌구먼!! 난 이거 잊을수없어,ㅋㅋ",1,재밌구먼 난 이거 잊을수없어ㅋㅋ,재밌다 난 이 거 잊다 없다 ㅋㅋ,재밌구먼 난 거 잊을수 없어
4,1053,1054,진짜뭐햐는 짓인지...스토리가 억지스럽고 자연스럽지못하고어처구니가 없네요,0,진짜뭐햐는 짓인지스토리가 억지스럽고 자연스럽지못하고어처구니가 없네요,진짜뭐햐 는 짓 인지 스토리 가 억지스럽다 자연스럽다 어처구니 가 없다,진짜뭐햐 짓 스토리 억지스럽고 자연스럽지못하고 어처구니 없네요


벡터화

In [27]:
X_train = train.main_pos
y_train = train.label

In [28]:
from sklearn.feature_extraction.text import CountVectorizer

In [29]:
vectorizer = CountVectorizer()
vectorizer.fit(X_train)
X_train_vec = vectorizer.transform(X_train)

모델 학습

In [30]:
from sklearn.linear_model import LogisticRegression 
model = LogisticRegression() 
model.fit(X_train_vec, y_train)

LogisticRegression()

모델 성능 검증

In [31]:
# Preprocessed
val['preprocessed'] = val['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
val['preprocessed'] = val['preprocessed'].str.replace(" +", " ")

# Tokenize
tokenized = []
for sentence in val['preprocessed']:
    tokens = okt.morphs(sentence, stem = True)
    tokenize = " ".join(tokens)
    tokenized.append(tokenize)
val["tokenized_stem"] = pd.DataFrame(tokenized)

# main_pos
main_pos = []
for sentence in val['document']:
    pos = okt.pos(sentence)
    main_words = [word_pos[0] for word_pos in pos if word_pos[1] in ("Noun", "Adverb", "Adjective", "Verb")]
    main_words_str = " ".join(main_words)
    main_pos.append(main_words_str)
val["main_pos"] = pd.DataFrame(main_pos)

val.head()

,index,id,document,label,preprocessed,tokenized_stem,main_pos
0,2092,2093,난왜별로지;; 다 팬심으로보는건가? 연기도어색하고 집중도안되던데 노잼,0,난왜별로지 다 팬심으로보는건가 연기도어색하고 집중도안되던데 노잼,난왜 별 로지 다 팬심 으로 보다 건가 연 기도 어색하다 집중 도 안되다 노잼,난왜 로지 다 팬심 보는 건가 기도 어색하고 집중 안되던데 노잼
1,712,713,가장 재밌게 본 영화가 뭐야? 라고 한다면 다섯손가락안에 꼽히는 영화,1,가장 재밌게 본 영화가 뭐야 라고 한다면 다섯손가락안에 꼽히는 영화,가장 재밌다 보다 영화 가 뭐 야 라고 하다 다섯손가락 안 에 꼽히다 영화,가장 재밌게 본 영화 뭐 한다면 다섯손가락 안 꼽히는 영화
2,3386,3387,이 영화가 개인적으론 우리나라 스포츠 휴먼 감동 영화의 OO점이라고 생각됨!,1,이 영화가 개인적으론 우리나라 스포츠 휴먼 감동 영화의 점이라고 생각됨,이 영화 가 개인 적 으론 우리나라 스포츠 휴먼 감동 영화 의 점 이라고 생각 되다,이 영화 개인 우리나라 스포츠 휴먼 감동 영화 점 생각 됨
3,4503,4504,왜명작이라불리우는지오늘보고알았다 진짜최고였다,1,왜명작이라불리우는지오늘보고알았다 진짜최고였다,왜 명작 이라 불리다 오늘 보고 알다 진짜 최고 이다,왜 명작 불리우는지 오늘 보고 알았다 진짜 최고 였다
4,2516,2517,프랑스 영화가 좋은 이유 하나 더 추가요~,1,프랑스 영화가 좋은 이유 하나 더 추가요,프랑스 영화 가 좋다 이유 하나 더 추가 요,프랑스 영화 좋은 이유 하나 더 추가


In [33]:
X_val = val.main_pos
y_val = val.label

X_val_vec = vectorizer.transform(X_val)

In [34]:
y_pred = model.predict(X_val_vec)
print(y_pred)

[0 1 1 ... 0 0 0]


In [35]:
from sklearn import metrics
print('accuracy =', metrics.accuracy_score(y_val, y_pred))

accuracy = 0.8208


### Test set

In [37]:
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ybsession/0210 KoNLPyWordcloud/test.csv')
test

,id,document
0,1,시간 때우기 좋은 영화 지루함
1,2,훈훈한 정이 느껴지는 영화! 가족끼리 드라마 보듯이 보면 딱~!
2,3,Childhood fantasy
3,4,멋있는 영화입니다. 잊을 수 없는!
4,5,너무 감동적이네요 펑펑 울었습니다
...,...,...
4995,4996,연계도 제대로 안되고 뭔가 부족한느낌이들고 찝찝한영화
4996,4997,내생에 최악의 영화중 하나 졸려서 미칠뻔....
4997,4998,지금까지 나온 드라마 중 최고였습니다. 100점 만점!
4998,4999,영화값으로 엄청난 공연을 보았다 - 이게 웬 횡재 ..


In [38]:
test.isna().sum()

id          0
document    0
dtype: int64

In [39]:
# Preprocessed
test['preprocessed'] = test['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") 
test['preprocessed'] = test['preprocessed'].str.replace(" +", " ")

# Tokenize
tokenized = []
for sentence in test['preprocessed']:
    tokens = okt.morphs(sentence, stem = True) 
    tokenize = " ".join(tokens)
    tokenized.append(tokenize)
test["tokenized_stem"] = pd.DataFrame(tokenized)

# main_pos
main_pos = []
for sentence in test['document']:
    pos = okt.pos(sentence)
    main_words = [word_pos[0] for word_pos in pos if word_pos[1] in ("Noun", "Adverb", "Adjective", "Verb")]
    main_words_str = " ".join(main_words)
    main_pos.append(main_words_str)
test["main_pos"] = pd.DataFrame(main_pos)

test.head()

,id,document,preprocessed,tokenized_stem,main_pos
0,1,시간 때우기 좋은 영화 지루함,시간 때우기 좋은 영화 지루함,시간 때우다 좋다 영화 지루함,시간 때우기 좋은 영화 지루함
1,2,훈훈한 정이 느껴지는 영화! 가족끼리 드라마 보듯이 보면 딱~!,훈훈한 정이 느껴지는 영화 가족끼리 드라마 보듯이 보면 딱,훈훈하다 정이 느껴지다 영화 가족 끼리 드라마 보다 보다 딱,훈훈한 정이 느껴지는 영화 가족 끼리 드라마 보듯이 보면 딱
2,3,Childhood fantasy,,,
3,4,멋있는 영화입니다. 잊을 수 없는!,멋있는 영화입니다 잊을 수 없는,멋있다 영화 이다 잊다 수 없다,멋있는 영화 입니다 잊을 수 없는
4,5,너무 감동적이네요 펑펑 울었습니다,너무 감동적이네요 펑펑 울었습니다,너무 감동 적다 펑펑 울다,너무 감동 적이네요 펑펑 울었습니다


In [40]:
X_test = test.main_pos
X_test_vec = vectorizer.transform(X_test)
pred_test = model.predict(X_test_vec)
print(pred_test)

[0 1 0 ... 1 1 1]


### 제출 파일

In [41]:
submission = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ybsession/0210 KoNLPyWordcloud/sample_submission.csv')
submission.head()

,id,label
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0


In [42]:
submission['label'] = pred_test

In [43]:
submission

,id,label
0,1,0
1,2,1
2,3,0
3,4,1
4,5,1
...,...,...
4995,4996,0
4996,4997,0
4997,4998,1
4998,4999,1


In [44]:
submission.to_csv("submission_baseline2.csv", index = False)